In [ ]:
### Data
import xarray as xr
import cfgrib
import numpy as np
from wrf import smooth2d

### Plotting
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')

def get_slp(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'meanSea','shortName': 'prmsl'}})
    var = file.get('prmsl').to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

def get_z(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa','shortName': 'gh'}})
    var = file.get('gh')[12].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var


In [ ]:
min_lat1 = 28
max_lat1 = 44
min_lon1 = -97
max_lon1 = -78

z_a = get_z('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111512.f000.grib2',
                 min_lat1, max_lat1, min_lon1, max_lon1)
z_f = get_z('/p/work1/lloveras/nov2018/gfs_files/forecast_2018111312/gfs.0p25.2018111312.f048.grib2',
                 min_lat1, max_lat1, min_lon1, max_lon1)

min_lat2 = 32
max_lat2 = 48
min_lon2 = -82
max_lon2 = -63

slp_a = get_slp('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111612.f000.grib2',
                 min_lat2, max_lat2, min_lon2, max_lon2)
slp_f = get_slp('/p/work1/lloveras/nov2018/gfs_files/forecast_2018111312/gfs.0p25.2018111312.f072.grib2',
                 min_lat2, max_lat2, min_lon2, max_lon2)


In [ ]:
fig, axd = plt.subplot_mosaic([['left','right'],['cbar1','cbar2']],
                              constrained_layout=True, figsize=(7.0,3.6), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1,0.05]},
                              per_subplot_kw={'left':{'projection':crs.PlateCarree()},
                                              'right':{'projection':crs.PlateCarree()}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

axd['left'].set_extent((min_lon1,max_lon1,min_lat1,max_lat1))

cs1 = axd['left'].contour(z_a['longitude'], z_a['latitude'], smooth2d(z_a['gh']/10.,1),
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs2 = axd['left'].contour(z_f['longitude'], z_f['latitude'], smooth2d(z_f['gh']/10.,1),
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['left'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['left'].contourf(z_a['longitude'], z_a['latitude'], z_f['gh'] - z_a['gh'],
                  levels=np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*15,extend='both',
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-95, -90, -85, -80])
gls.ylocator = mticker.FixedLocator([26, 31, 36, 41])

axd['left'].set_title('1200 UTC 15 Nov 2018 ($t$ = 48 h)')
axd['left'].text(0.0, 1.0, '(a) Z500',transform=axd['left'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### RIGHT
#############

axd['right'].set_extent((min_lon2,max_lon2,min_lat2,max_lat2))

cs1 = axd['right'].contour(slp_a['longitude'], slp_a['latitude'], slp_a['prmsl']/100.,
                  levels=np.arange(0,1500,6),
                  colors='k', transform=crs.PlateCarree())
axd['right'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='k')
cs2 = axd['right'].contour(slp_f['longitude'], slp_f['latitude'], slp_f['prmsl']/100.,
                  levels=np.arange(0,1500,6),
                  colors='magenta', transform=crs.PlateCarree())
axd['right'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='magenta')
im2 = axd['right'].contourf(slp_a['longitude'], slp_a['latitude'], (slp_f['prmsl'] - slp_a['prmsl'])/100.,
                  levels=np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*1.5,extend='both',
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['right'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-80, -75, -70, -65])
gls.ylocator = mticker.FixedLocator([30, 35, 40, 45])

axd['right'].set_title('1200 UTC 16 Nov 2018 ($t$ = 72 h)')
axd['right'].text(0.0, 1.0, '(b) SLP',transform=axd['right'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar1 = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar1'])
cbar1.set_ticklabels(['-135','','-75','','-15','15','','75','','135'],fontsize=10)

cbar2 = fig.colorbar(im2, orientation='horizontal', cax=axd['cbar2'])
cbar2.set_ticklabels(['-13.5','','-7.5','','-1.5','1.5','','7.5','','13.5'],fontsize=10)

plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig02_gfs_errors.pdf',bbox_inches='tight')
plt.show()
